In [ ]:
# Simulation to build a Probability Density Distribution (PDF) of losses in a credit portfolio.

# We start by defining the correlation matrix of defaults for three categories of companies.

import numpy as np

correlation_matrix = np.array([[1,0.2981, 0.5345],
                               [0.2981, 1, 0.5345],
                               [0.5345, 0.5345, 1]])        

# We use a Cholesky decomposition to generate correlated random variables.
from scipy.linalg import cholesky

L = cholesky(correlation_matrix)
# We generate 3 default points via a normal distribution.
# This allows us to simulate correlated random variables using Cholesky decomposition.
# If Z is a standard normal random variable, then the correlated random variable X is given by X = LZ, where L is the Cholesky matrix of the covariance matrix.

from scipy.stats import norm

# Dpoint is used to obtain default thresholds for the three categories of companies.
# Default risk at 10%, 20%, and 90%.
Dpoint = np.array([norm.ppf(0.1), norm.ppf(0.2), norm.ppf(0.9)])
# norm.ppf() represents the quantile function of the normal distribution, which is the inverse of the normal cumulative distribution function.
# norm.ppf(0.1) returns the value of the normal random variable that corresponds to 10% of the cumulative distribution.

# Parameters 
EAD = [1000, 7000, 4000] # Exposure at Default. The amount the company loses in case of default.
LGD = 0.2 # Average Loss Given Default. The percentage of the EAD that is lost to the bank in case of default.
Loss_distribution = [] # List to store total losses

# We generate correlated default simulations using a Monte-Carlo method.

def Monte_Carlo(n):
    for i in range(n):
        # We generate correlated random variables.
        Z = np.random.normal(size=(3,)) # Uncorrelated, standard normal random variables
        X = L @ Z  # Matrix product with the Cholesky matrix to obtain correlated variables
    
        # We calculate the total losses
        total_loss = 0
        for j in range(3):
            if X[j] < Dpoint[j]:  # If the random variable is less than the default threshold
                total_loss += EAD[j] * LGD  # We add the corresponding loss
        
        Loss_distribution.append(total_loss)

Monte_Carlo(10000)  # We perform 10000 simulations

# We display the loss distribution   

import matplotlib.pyplot as plt

plt.hist(Loss_distribution, bins=30, density=True, alpha=0.6, color='g')
plt.title('Distribution of Total Losses')
plt.xlabel('Total Losses')
plt.ylabel('Probability Density')
plt.grid(True)
plt.show()

# We calculate the Value at Risk (VaR) at 95% and 99% of the loss distribution
VaR_95 = np.percentile(Loss_distribution, 95)
VaR_99 = np.percentile(Loss_distribution, 99)
print(f"VaR at 95%: {VaR_95}")
print(f"VaR at 99%: {VaR_99}")

# We calculate the Expected Loss (EL) of the loss distribution
EL = np.mean(Loss_distribution)
print(f"Expected Loss (EL): {EL}")
# We calculate the Unexpected Loss (UL) of the loss distribution
UL = np.std(Loss_distribution)
print(f"Unexpected Loss (UL): {UL}")
# We calculate the Solvency Ratio of the bank
capital = 100000  # Bank's capital
solvency_ratio = capital / (EL + UL)
print(f"Solvency Ratio: {solvency_ratio}")
# We calculate the Loss Coverage Ratio of the bank
coverage_ratio = capital / EL
print(f"Loss Coverage Ratio: {coverage_ratio}")
# We calculate the Credit Risk Ratio of the bank
credit_risk_ratio = (EL + UL) / capital
print(f"Credit Risk Ratio: {credit_risk_ratio}")
# We calculate the Loss on Default Ratio of the bank
loss_on_default_ratio = EL / (EL + UL)
print(f"Loss on Default Ratio: {loss_on_default_ratio}")
